In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
import json
f = open('/content/drive/My Drive/cns/data.json',)
data = json.load(f)
f.close()

In [0]:
mails = pd.read_csv("/content/drive/My Drive/spam_ham_dataset.csv", header=0)
mails.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [0]:
data_train, data_test, label_train, label_test = train_test_split(mails['text'], mails['label'],test_size=.10)

In [0]:
tagged_mails = []
for mail_text, label in zip(data_train, label_train):
  tagged_mail = TaggedDocument(words=word_tokenize(mail_text.lower()), tags=[label])
  tagged_mails.append(tagged_mail) 

In [0]:
stop_words = set(stopwords.words('english')) 

In [0]:
max_epochs = 50
vec_size = 100
alpha_val = 0.025
min_word_count = 1

In [0]:
model = Doc2Vec(vector_size=vec_size, alpha=alpha_val, 
                min_count=min_word_count)
model.build_vocab(tagged_mails)
model.train(tagged_mails, total_examples=model.corpus_count,
             epochs=max_epochs)

In [0]:
model.save("/content/drive/My Drive/cns/d2v.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
mail_vectors = []
count_list_train = []
for text, label in zip(data_train, label_train):
  mail_text = word_tokenize(text.lower())
  count = 0
  for word in mail_text:
    if word in stop_words:
      continue
    if not word.isalpha():
      continue
    if len(word) <= 1:
      continue
    if word in data:
      count=count+1
  count_list_train.append(count)
  mail_vector = model.infer_vector(mail_text)
  mail_vectors.append(mail_vector)

In [0]:
test_vectors = []
count_list_test = []
for text, label in zip(data_test, label_test):
  test_text = word_tokenize(text.lower())
  count = 0
  for word in test_text:
    if word in stop_words:
      continue
    if not word.isalpha():
      continue
    if len(word) <= 1:
      continue
    if word in data:
      count=count+1
  count_list_test.append(count)
  test_vector = model.infer_vector(test_text)
  test_vectors.append(test_vector)

In [0]:
df = pd.DataFrame(data=mail_vectors)
df['spamcount']=count_list_train
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,spamcount
0,-0.512290,0.355454,1.548711,2.621447,1.509517,-0.632970,-0.348533,-0.469144,0.308829,-1.106185,0.583526,-0.580980,0.079963,-0.462139,0.983848,2.356924,-0.905652,2.575897,-1.898740,0.420174,-0.252343,-1.381593,-0.140399,-0.093044,-0.745450,1.917677,-0.330067,-0.650455,-0.707446,0.447406,-0.178087,0.800016,-2.721414,-0.509425,0.420353,0.580390,-0.452517,0.105598,0.653302,0.734411,...,-0.072457,-2.354902,0.220491,-0.274987,-0.744270,-0.271797,0.339457,-1.061891,-1.037638,-1.057488,-1.170796,1.087121,-2.088859,0.690810,-2.440974,0.113271,0.448285,1.057281,-0.073559,-1.240923,1.044856,-0.754344,0.597701,-0.121452,-0.988892,-0.664971,-0.657560,0.314032,0.369980,-0.198629,-0.711329,1.593778,1.766027,-0.579768,0.385840,-1.192124,-0.781490,1.022806,0.638392,0
1,-0.854242,0.723655,1.918259,2.553410,1.449654,-1.305314,-0.638062,-0.137289,0.506380,-0.766705,-0.395015,-0.131414,-0.274506,-0.819274,1.698029,3.675877,-0.441504,2.651220,-1.796151,1.029708,0.791592,-1.275150,-0.355202,-0.433953,-0.099633,1.444499,-1.303706,0.166375,0.014966,0.952160,-0.763666,1.326383,-1.309205,-0.925666,0.824360,0.578143,-0.658329,-1.498629,0.709154,2.561542,...,0.871371,-3.265822,1.143881,-0.452904,-0.718110,-0.174085,0.031197,-0.441968,-1.764333,-1.318273,-0.810899,1.697459,-1.829338,0.935783,-2.522442,-0.465772,-0.123708,0.698862,0.361567,-1.132301,1.521024,-1.034016,0.962160,-0.188357,-0.826822,-0.941701,-0.807800,0.414989,-0.074749,0.212520,0.800028,3.593783,1.022615,0.446492,0.208826,-1.613355,-0.834022,1.135509,1.223292,3
2,-0.017959,0.089357,0.117201,0.040114,0.111443,0.078356,-0.007025,-0.053249,0.044376,-0.119792,-0.020706,-0.067252,0.002877,0.066274,0.032882,0.139324,-0.111801,0.103482,-0.132817,0.075219,-0.027398,-0.121496,-0.011748,-0.030022,-0.061054,0.124887,-0.008161,0.063965,0.009868,0.054348,0.037083,0.015651,-0.091025,0.033287,-0.052703,-0.014109,-0.023686,0.035184,-0.034852,-0.011906,...,-0.037652,-0.144212,-0.053679,0.031312,-0.014647,0.013996,0.049067,0.027110,-0.091305,-0.093099,-0.043179,-0.014461,-0.087332,0.041319,-0.116966,-0.008408,-0.027648,0.042194,-0.031264,-0.054296,0.137680,-0.009367,0.053645,-0.000191,-0.049132,-0.012801,-0.014871,0.021007,0.003220,0.026569,-0.062633,0.137491,0.057225,-0.036948,-0.066389,0.011090,-0.077226,0.048245,-0.023910,0
3,-0.267087,-0.081463,1.526793,5.581871,2.141657,-1.138260,-0.628747,-1.231510,1.977034,-0.983033,-1.660974,-1.293509,0.462910,-1.967057,2.616373,3.271384,-1.560908,6.027159,0.052463,2.102479,0.467957,-2.334594,0.518371,-1.087799,-1.210132,1.525529,-2.624497,-0.088381,-0.411070,1.127367,-1.859348,1.217549,-3.364122,0.513815,1.229165,0.897555,-1.155884,0.185954,0.104174,3.255559,...,1.542688,-3.497344,0.734992,-1.487720,-1.079431,0.452110,1.039383,-2.849725,-1.423122,-2.384061,-2.765774,1.653593,-3.042495,1.961288,-4.449443,0.388817,2.024677,1.697304,1.150392,-2.864448,2.908722,-1.451178,1.890843,-1.034073,-1.371400,-3.021541,-0.473560,1.485029,0.036560,-0.178517,-1.721780,4.018937,4.176904,0.987622,0.547578,-0.300526,-1.354586,3.009374,1.584697,45
4,0.033949,0.631008,1.681877,3.986263,2.541939,-2.268687,-0.439932,-0.180924,1.046483,-1.852831,-0.312084,-0.050405,-1.280542,-1.070219,1.447699,6.563251,-2.054327,4.410519,-4.205749,3.900157,1.236331,-0.397794,1.413858,-2.169598,-2.679106,2.298197,-2.202069,-0.108035,-0.491550,1.865787,-1.120839,0.319424,-4.522634,-0.521013,0.956808,2.184215,-2.016062,-2.300021,0.852781,3.293898,...,0.458038,-3.484695,-0.069621,-1.665189,-1.527320,-0.515428,1.513421,0.173118,-1.695424,-4.471043,-2.889209,1.961306,-3.370040,2.101297,-3.380214,-1.772511,0.906252,-0.133706,1.903615,-3.433765,2.329841,-2.276058,3.447656,-1.065941,-2.974558,-2.269061,-0.585376,1.472075,0.368135,0.365547,-0.578897,4.752114,2.010721,0.660698,0

In [0]:
df_test=pd.DataFrame(data=test_vectors)
df_test['spamcount']=count_list_test
df_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,spamcount
0,-0.687620,-1.735716,3.737378,1.697027,1.907215,-2.504322,-0.150281,-2.535871,-1.797461,-1.100084,-2.333880,-1.085382,0.302546,-2.672055,1.605211,-0.054575,-2.558499,3.047375,1.141280,2.667225,-0.172242,-1.372850,1.084451,0.278657,-1.370121,-1.114139,-2.348778,2.277155,0.929687,1.828466,-3.809859,-0.228464,-1.276260,0.611983,2.072614,1.527362,-0.717691,1.920808,3.150257,2.056272,...,0.436288,-0.874443,-0.269661,0.428749,0.589049,-0.808156,1.967851,0.430723,-1.565806,-0.960887,-0.570193,0.255586,-0.353319,-2.096871,-5.130882,-1.399668,0.921536,2.620809,-1.756779,-4.219673,1.612398,-2.091354,1.115474,-0.083025,0.015115,-1.252007,-1.200863,0.823978,-2.138819,-0.559353,-1.006610,3.818496,3.966760,-0.147937,0.893437,-0.738856,0.886894,1.421004,-0.104650,30
1,-2.927333,0.712543,0.701929,1.646807,-0.789651,1.058187,0.582811,0.242197,0.819735,-1.803071,0.279511,-0.621580,0.389297,-2.277933,0.843441,1.478370,-1.374890,4.470685,-2.353151,2.289753,2.850330,-2.308969,-0.424786,0.232738,-0.729757,0.577606,1.590719,-0.617818,0.238209,0.108915,-0.632795,0.389036,-4.310705,-0.443756,1.190686,-0.394076,-1.098996,1.346688,2.301074,3.607662,...,1.425393,-2.714524,1.033071,0.005081,-1.807383,1.322586,0.398335,-3.026596,-1.019853,-4.072007,0.493710,1.409170,-0.645120,0.892124,-3.074058,-1.257971,-1.530746,1.474721,-1.856625,-0.412228,-0.302079,-2.742420,0.607751,0.798504,0.890056,-0.170352,-1.524403,-0.775687,0.702396,-0.070683,-3.564084,1.137950,2.666305,1.282224,-1.814270,-0.749794,-1.280137,2.001190,2.970916,6
2,0.118781,1.971752,0.441408,0.145629,3.032709,-1.024908,-0.222374,-1.473283,0.316930,-2.465489,-0.446438,0.972043,-1.586874,-1.000761,0.784129,3.580055,-0.813784,1.362910,-2.834284,3.795583,1.388234,-2.646870,0.371156,0.254633,-0.736646,1.078805,-0.114463,-0.413620,-1.158861,0.947971,-1.590566,0.077003,-3.051413,1.349188,2.407990,1.136053,-1.416121,-2.005646,-0.615861,1.468301,...,-0.120991,-2.429377,0.029728,-1.286411,-2.585633,-1.482040,1.215150,-1.003012,-0.901114,-1.442410,-1.066835,3.415456,-1.681756,1.276662,-1.134336,-1.354348,1.239298,1.635917,0.615127,-2.993619,-1.163558,-1.078509,0.557869,0.947205,0.470873,-1.112416,-0.318699,0.261294,0.032754,-0.078569,-2.108848,2.172490,1.550791,1.368679,-0.024048,-0.199210,0.531511,1.484709,1.693259,16
3,-2.070490,-0.046501,2.153018,3.308307,2.523893,-1.706300,1.607341,1.003854,-1.174512,0.625193,-1.682228,-2.730727,-1.231881,-1.404216,0.354550,1.457649,-3.087457,2.389232,-0.289087,0.873175,-0.352865,-2.205444,0.504413,0.094764,-1.458212,0.610578,-0.975986,0.818893,0.278969,-0.339626,-3.705747,-0.829006,-2.310191,1.237682,1.488014,2.059154,-2.474417,1.211045,2.263678,0.647971,...,0.139612,-1.565585,0.042533,0.929861,0.556794,-0.709004,-0.066874,0.192359,-1.397785,1.511680,-1.279739,0.723337,-0.704317,0.481355,-1.735079,-0.805866,2.446211,1.902840,1.892025,-1.509044,0.883533,1.055749,0.875492,-0.786966,-0.261359,-2.567873,-0.920249,1.736438,-0.625045,-1.475804,-1.959882,2.347156,2.030386,0.827845,-2.830603,-1.301537,-0.356154,0.174779,-0.959442,9
4,-0.679758,-0.743099,0.833788,1.935166,-1.441310,-0.028323,0.687804,-3.045657,-1.200159,-3.228300,-2.613025,1.235637,-0.252343,0.754460,0.337069,1.847785,-2.426219,1.755285,0.071489,0.613530,0.709246,-0.879087,3.976319,-1.689540,-2.724398,-0.997454,0.873175,0.148332,-2.483880,-0.849326,0.539874,0.296446,-2.551581,-0.492648,-0.761158,0.405118,-0.665951,-2.001734,2.896796,5.046061,...,0.001759,-1.614900,0.406760,-1.621342,1.396817,-2.240496,-1.518907,-1.251776,1.087833,-1.734644,-2.853049,2.776491,0.224487,1.941198,-3.175600,-0.465385,-0.303284,2.275928,-2.162595,-3.363383,2.215513,-1.164366,2.625828,-1.291364,-2.788356,0.808839,-1.163956,0.525678,-1.220586,1.106064,-1.474895,0.206342,3.346082,0.320469,0.813926,-

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [0]:
rf = RandomForestClassifier(max_depth=10, random_state=0)
rf.fit(df,label_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
lr = LogisticRegression(random_state=0).fit(df,label_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
sv = SVC()
sv.fit(df,label_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
test_predict_rf = rf.predict(df_test)
accuracy_score(test_predict_rf,label_test)

0.9305019305019305

In [0]:
test_predict_lr = lr.predict(df_test)
accuracy_score(test_predict_lr,label_test)

0.9575289575289575

In [0]:
test_predict_sv = sv.predict(df_test)
accuracy_score(test_predict_sv,label_test)

0.9498069498069498

In [0]:
correctly_classified = 0
incorrect_spam = 0
incorrect_ham = 0
for text, label in zip(data_test, label_test):
  mail_text = word_tokenize(text.lower())
  mail_vector = model.infer_vector(mail_text)
  predicted_label = model.docvecs.most_similar([mail_vector], topn = 1)
  predicted_label = predicted_label[0][0]
  if predicted_label == label:
    correctly_classified += 1
  elif label == "spam":
    incorrect_spam += 1
  else:
    incorrect_ham += 1


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
print("Correct = " + str(correctly_classified))
print("Wrong Spam = " + str(incorrect_spam))
print("Wrong Ham = " + str(incorrect_ham))

Correct = 427
Wrong Spam = 89
Wrong Ham = 2
